CoNLL-2003 dataset

In [14]:
from datasets import load_dataset

raw_datasets = load_dataset("conll2003")

raw_datasets["train"][0]["tokens"]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [2]:
ner_feature = raw_datasets["train"].features["ner_tags"]
label_names = ner_feature.feature.names

label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [3]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""

for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


Processing

In [4]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [5]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()

print(labels)
print(word_ids)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [6]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [7]:
tokenized_datasets = raw_datasets.map(tokenize_and_align_labels,
                                      batched=True,
                                      remove_columns=raw_datasets["train"].column_names)

Fine-tuning the model with Trainer API

In [8]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])

batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [9]:
import evaluate

metric = evaluate.load("seqeval")
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [10]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

Import model itself

In [24]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, 
                                                        id2label=id2label, 
                                                        label2id=label2id)

model.config.num_labels

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


9

Fine-tune with Trainer API

In [12]:
from huggingface_hub import notebook_login

notebook_login()

In [16]:
from transformers import TrainingArguments
from transformers import Trainer

# import os
# os.environ["NCCL_DEBUG"] = "INFO"

training_args = TrainingArguments(
    "bert-finetuned-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
    hub_model_id="HarryDn/huggingface-nlp-bert-finetuned-ner",
    no_cuda=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
)

trainer.train()

/media/gleb/Data/Repos/huggingface-main-nlp-tasks/.venv/lib/python3.12/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.000000,nan,0.000000,0.000000,0.000000,0.761214
2,0.000000,nan,0.000000,0.000000,0.000000,0.761214
3,0.000000,nan,0.000000,0.000000,0.000000,0.761214


/media/gleb/Data/Repos/huggingface-main-nlp-tasks/.venv/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/gleb/Data/Repos/huggingface-main-nlp-tasks/.venv/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/media/gleb/Data/Repos/huggingface-main-nlp-tasks/.venv/lib/python3.12/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st

TrainOutput(global_step=5268, training_loss=0.0, metrics={'train_runtime': 211.4467, 'train_samples_per_second': 199.213, 'train_steps_per_second': 24.914, 'total_flos': 920771584279074.0, 'train_loss': 0.0, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub(commit_message="Retrained with default parameters")

Custom training loop

In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    dataset=tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

eval_dataloader = DataLoader(
    dataset=tokenized_datasets["validation"],
    collate_fn=data_collator,
    batch_size=8,
)

In [18]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [19]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [20]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [22]:
from huggingface_hub import Repository

repo_name = "HarryDn/huggingface-nlp-bert-finetuned-ner"
output_dir = "bert-finetuned-ner-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/media/gleb/Data/Repos/huggingface-main-nlp-tasks/.venv/lib/python3.12/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/HarryDn/huggingface-nlp-bert-finetuned-ner into local empty directory.


Download file model.safetensors:   0%|          | 24.0k/411M [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 5.24k/5.24k [00:00<?, ?B/s]

Clean file training_args.bin: 100%|##########| 5.24k/5.24k [00:00<?, ?B/s]

Clean file model.safetensors:  35%|###4      | 143M/411M [00:00<?, ?B/s]

In [25]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    return true_labels, true_predictions

In [27]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

# Move model to the correct device
model = model.to(accelerator.device)

for epoch in range(num_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        batch = {key: value.to(accelerator.device) for key, value in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        batch = {key: value.to(accelerator.device) for key, value in batch.items()}

        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch["labels"]
        
        # Necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    eval_metric = metric.compute()
    print(f"Epoch {epoch}: {eval_metric}")


  0%|          | 0/5268 [00:00<?, ?it/s]

Epoch 0: {'LOC': {'precision': 0.0522591181273816, 'recall': 0.009097801364670205, 'f1': 0.015497618855436273, 'number': 10552}, 'MISC': {'precision': 0.010845986984815618, 'recall': 0.004777830864787387, 'f1': 0.006633499170812605, 'number': 2093}, 'ORG': {'precision': 0.08351976137211037, 'recall': 0.005251066622907778, 'f1': 0.009880899867666519, 'number': 21329}, 'PER': {'precision': 0.014657980456026058, 'recall': 0.003991720875221762, 'f1': 0.006274692075296305, 'number': 6764}, 'overall_precision': 0.041231908448333895, 'overall_recall': 0.0060140409445726345, 'overall_f1': 0.010497000856898028, 'overall_accuracy': 0.06194442809207041}
Epoch 1: {'LOC': {'precision': 0.0522591181273816, 'recall': 0.009097801364670205, 'f1': 0.015497618855436273, 'number': 10552}, 'MISC': {'precision': 0.010845986984815618, 'recall': 0.004777830864787387, 'f1': 0.006633499170812605, 'number': 2093}, 'ORG': {'precision': 0.08351976137211037, 'recall': 0.005251066622907778, 'f1': 0.00988089986766651